In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten
from tensorflow.keras.models import Model

C:\Users\Talha Dar\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv('/kaggle/input/cat-diseases/Cat dieases.csv', encoding='utf-8', header=0) 

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/cat-diseases/Cat dieases.csv'

In [ ]:
data.head()

In [ ]:
data.columns = data.columns.str.strip()

In [ ]:
print(data.columns)
print(data.head())

In [ ]:
print(data.columns.tolist())

In [ ]:
#data['Symptoms'] = data['Symptoms'].str.lower().str.replace('[^\w\s]', '')
#data['Cat Diseases'] = data['Cat Diseases'].str.lower()
#data['Medication'] = data['Medication'].str.lower()

In [ ]:
symptoms = data['Symptoms']
diseases = data['Cat Diseases']
medications = data['Medication']

In [ ]:
# Create mappings for symptoms to diseases and medications
#symptoms_to_diseases = dict(zip(data['Symptoms'], data['Cat Diseases']))
#symptoms_to_medications = dict(zip(data['Symptoms'], data['Medication']))

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(symptoms)
symptom_sequences = tokenizer.texts_to_sequences(symptoms)
symptom_sequences_padded = pad_sequences(symptom_sequences, padding='post')

In [ ]:
# Convert diseases and medications to numerical labels
disease_labels = pd.factorize(diseases)[0]
medication_labels = pd.factorize(medications)[0]

In [ ]:
# Create model architecture
input_layer = Input(shape=(symptom_sequences_padded.shape[1],))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)(input_layer)
flatten_layer = Flatten()(embedding_layer)
shared_dense_layer = Dense(128, activation='relu')(flatten_layer)
disease_output = Dense(len(np.unique(disease_labels)), activation='softmax')(shared_dense_layer)
medication_output = Dense(len(np.unique(medication_labels)), activation='softmax')(shared_dense_layer)

model = Model(inputs=input_layer, outputs=[disease_output, medication_output])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(symptom_sequences_padded, [disease_labels, medication_labels], epochs=30, batch_size=32)

In [ ]:
# Example prediction
new_symptom = "Itching, redness, skin rashes"
new_symptom_sequence = tokenizer.texts_to_sequences([new_symptom])
new_symptom_padded = pad_sequences(new_symptom_sequence, maxlen=symptom_sequences_padded.shape[1], padding='post')
predicted_disease, predicted_medication = model.predict(new_symptom_padded)

In [ ]:
# Convert predicted indices back to disease and medication labels
predicted_disease_label = np.argmax(predicted_disease)
predicted_medication_label = np.argmax(predicted_medication)

# Convert numerical labels back to original disease and medication names
predicted_disease_name = pd.unique(diseases)[predicted_disease_label]
predicted_medication_name = pd.unique(medications)[predicted_medication_label]

print("Predicted Disease:", predicted_disease_name)
print("Predicted Medication:", predicted_medication_name)